<a href="https://colab.research.google.com/github/Tanish-analyst/financial-sentiment-analyzer/blob/main/Source%20code/fine_tuned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("/content/chatgpt senti.xlsx")
df

,headlines,sentiment
0,Company profits surge 25 percent in latest qua...,positive
1,Stock market hits record high amid investor op...,positive
2,Federal Reserve signals confidence in economic...,positive
3,Unemployment rate drops to lowest level in a d...,positive
4,Tech sector leads Wall Street rally with stron...,positive
...,...,...
2995,Global trade organization maintains current ta...,neutral
2996,Business mentorship program expands outreach e...,neutral
2997,Healthcare information systems firm enhances i...,neutral
2998,Industrial tech manufacturer maintains robotic...,neutral


In [ ]:
label_mapping = {"neutral": 0, "positive": 1, "negative": 2}
df['sentiment'] = df['sentiment'].map(label_mapping)
df

,headlines,sentiment
0,Company profits surge 25 percent in latest qua...,1
1,Stock market hits record high amid investor op...,1
2,Federal Reserve signals confidence in economic...,1
3,Unemployment rate drops to lowest level in a d...,1
4,Tech sector leads Wall Street rally with stron...,1
...,...,...
2995,Global trade organization maintains current ta...,0
2996,Business mentorship program expands outreach e...,0
2997,Healthcare information systems firm enhances i...,0
2998,Industrial tech manufacturer maintains robotic...,0


In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset=dataset.shuffle(seed=42)

In [ ]:
dataset

Dataset({
    features: ['headlines', 'sentiment'],
    num_rows: 3000
})

In [ ]:
dataset = dataset.rename_column("sentiment", "label")

In [ ]:
split_dataset = dataset.train_test_split(test_size=0.2)

# Access the train and test datasets
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [ ]:
train_dataset,test_dataset

(Dataset({
     features: ['headlines', 'label'],
     num_rows: 2400
 }),
 Dataset({
     features: ['headlines', 'label'],
     num_rows: 600
 }))

In [ ]:
from transformers import AutoTokenizer

model_name = "yiyanghkust/finbert-tone"  # Or any other model
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["headlines"], truncation=True, padding="max_length")

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['headlines', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2400
})

In [ ]:
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "label"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "label"])

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True
)


In [ ]:
%pip install evaluate
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00


In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-26-4f3a1ecb8afe>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rajualbum295 (rajualbum295-maharaja-surajmal-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.108795,0.963333
2,No log,0.136546,0.963333
3,No log,0.149948,0.970000


TrainOutput(global_step=450, training_loss=0.16506856282552082, metrics={'train_runtime': 164.357, 'train_samples_per_second': 43.807, 'train_steps_per_second': 2.738, 'total_flos': 43783717168800.0, 'train_loss': 0.16506856282552082, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/kkkkkjjjjjj/results/commit/0f934f13d8e8bb30bd78d715f29f9d3e4fa4de8d', commit_message='End of training', commit_description='', oid='0f934f13d8e8bb30bd78d715f29f9d3e4fa4de8d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kkkkkjjjjjj/results', endpoint='https://huggingface.co', repo_type='model', repo_id='kkkkkjjjjjj/results'), pr_revision=None, pr_num=None)